In [43]:
import numpy as np
import pandas as pd
from sklearn.ensemble._iforest import _average_path_length


def diffi_score(forest, X, inlier_samples="auto"):
    pred = forest.predict(X)
    X_out = X[pred < 0]
    X_in = X[pred > 0]

    if inlier_samples == "all":
        k = X_in.shape[0]
    elif inlier_samples == "auto":
        k = X_out.shape[0]
    else:
        k = int(inlier_samples)
    if k < X_in.shape[0]:
        breakpoint()
        X_in = X_in.iloc[np.random.choice(X_in.shape[0], k, replace=False), :]

    return (_mean_cumulative_importance(forest, X_out) /
            _mean_cumulative_importance(forest, X_in))


def _mean_cumulative_importance(forest, X):
    '''
    Computes mean cumulative importance for every feature of given forest on dataset X
    '''

    f_importance = np.zeros(X.shape[1])
    f_count = np.zeros(X.shape[1])

    if forest._max_features == X.shape[1]:
        subsample_features = False
    else:
        subsample_features = True

    for tree, features in zip(forest.estimators_, forest.estimators_features_):
        X_subset = X[:, features] if subsample_features else X

        importance_t, count_t = _cumulative_ic(tree, X_subset)

        if subsample_features:
            f_importance[features] += importance_t
            f_count[features] += count_t
        else:
            f_importance += importance_t
            f_count += count_t

    return f_importance / f_count


def _cumulative_ic(tree, X):
    '''
    Computes importance and count for every feature of given tree on dataset X
    '''
    importance = np.zeros(X.shape[1])
    count = np.zeros(X.shape[1])

    node_indicator = tree.decision_path(X)
    node_loads = np.array(node_indicator.sum(axis=0)).reshape(-1)
    # depth is number of edges in path, same as number of nodes in path -1
    depth = np.array(node_indicator.sum(axis=1), dtype=float).reshape(-1) - 1
    # when the tree is pruned (i.e. more than one instance at the leaf)
    # we consider the average path length to adjust depthה
    leaves_index = tree.apply(X)
    depth += _average_path_length(node_loads[leaves_index])

    iic = _induced_imbalance_coeff(tree, X, node_loads)
    rows, cols = node_indicator.nonzero()
    for i, j in zip(rows, cols):
        f = tree.tree_.feature[j]
        # ignore leaf nodes
        if f < 0:
            continue
        count[f] += 1
        importance[f] += iic[j] / depth[i]

    return importance, count

def _induced_imbalance_coeff(tree, X, node_loads):
    '''
    Computes imbalance coefficient for every *node* of a tree on dataset X
    '''
    # epsilon as defined in the original paper
    _EPSILON = 1e-2
    iic = np.zeros_like(node_loads)
    for i in range(len(iic)):
        # ignore leaf nodes
        if tree.tree_.children_left[i] < 0:
            continue
        n_left = node_loads[tree.tree_.children_left[i]]
        n_right = node_loads[tree.tree_.children_right[i]]
        if n_left == 0 or n_right == 0:
            iic[i] = _EPSILON
            continue
        if n_left == 1 or n_right == 1:
            iic[i] = 1
            continue
        iic[i] = max(n_left, n_right) / node_loads[i]
    return iic

In [59]:
import numpy as np
import multiprocessing
from functools import partial

def get_support(data,feature_id,feature_val,cluster):
    """This function compute support for a given value
    """
    n_cluster_size = len(cluster)
    num = 0
    for j in range(n_cluster_size):
        if data[cluster[j],feature_id] == feature_val:
            num = num+1
    return num

def similarity_instance_cluster(data,instance_id,cluster):
    """This function computes the similarity between a new instance
    data[instance_id] and a cluster specified by cluster_id

    Parameters
    ----------
    data: array, shape(n_instances,n_features)
        matrix containing original data

    instance_id: int
        row number of the new instance

    cluster: list
        a list containing the ids of instances in this cluster
    
    Returns
    -------
    sim: float
        the similarity between the input instance and input cluster
    """
    n_instances,n_features = data.shape
    sim = 0.0

    for i in range(n_features):
        
        unique = []
        for j in range(len(cluster)):
            if data[cluster[j],i] not in unique:
                unique.append(data[cluster[j],i])
        temp = 0
        for j in range(len(unique)):
            temp = temp+get_support(data,i,unique[j],cluster)
        sim = sim+get_support(data,i,data[instance_id,i],cluster)*1.0/temp
    return sim

def squeezer(data,thre):
    """This function implements squeezer algorithm base on the paper "Squezzer
    : An Efficient Algorithm for Clustering Categorical Data"
    
    Parameters
    ----------
    data: array, shape(n_instances,n_features)
        the original data that need to be clustered, note that we donnot have
        to specify the number of clusters here

    thre: threshold used to decide if creating a new cluster is necessary

    Returns
    -------
    label: list, length(n_instances)
        label for every instance, label is a list of lists,list[i] represents
        cluster i, list[i] is a list containing the instances ID of cluster i
    """
    # Initialize the clustering result
    label = [[0]]
    
    # Obtain the number of instances and features from input data
    n_instances,n_features = data.shape
    print(f'num of instances: {n_instances}')
    for i in range(1,n_instances):
        print(f'instance {i}')
        # Current number of clusters
        n_clusters = len(label)
        sim = [0]*n_clusters
        # Compute similarity between data[i,:] and each cluster
        for j in range(n_clusters):
            sim[j] = similarity_instance_cluster(data,i,label[j])
        
        sim_max = max(sim)

        for j in range(n_clusters):
            if sim[j] == sim_max:
                sim_max_cluster_id = j

        if sim_max>=thre:
            label[sim_max_cluster_id].append(i)
        else:
            label.append([i])

    return label


def squeezer_parallel(data, thre):
    """This function implements squeezer algorithm base on the paper "Squezzer
    : An Efficient Algorithm for Clustering Categorical Data", with parallelization.

    Parameters
    ----------
    data: array, shape(n_instances,n_features)
        the original data that need to be clustered, note that we donnot have
        to specify the number of clusters here

    thre: threshold used to decide if creating a new cluster is necessary

    Returns
    -------
    label: list, length(n_instances)
        label for every instance, label is a list of lists,list[i] represents
        cluster i, list[i] is a list containing the instances ID of cluster i
    """
    # Initialize the clustering result
    label = [[0]]

    # Obtain the number of instances and features from input data
    n_instances, n_features = data.shape
    print(f'num of instances: {n_instances}')

    # Create a pool of workers
    pool = multiprocessing.Pool()

    for i in range(1, n_instances):
        print(f'instance {i}')
        # Current number of clusters
        n_clusters = len(label)

        # Compute similarity between data[i,:] and each cluster in parallel
        func_partial = partial(similarity_instance_cluster, data, i)
        sim = pool.map(func_partial, [label[j] for j in range(n_clusters)])

        sim_max = max(sim)
        sim_max_cluster_id = sim.index(sim_max)

        if sim_max >= thre:
            label[sim_max_cluster_id].append(i)
        else:
            label.append([i])

    # Close the pool of workers
    pool.close()
    pool.join()

    return label

In [45]:
#load dataset
# df = pd.read_csv('pii_financial_dataset_with_embeddings.csv')
# y = df.Performance
# X = df.drop(columns=['Target', 'Performance', 'index'])

In [46]:
FILE_PATH = "fairness_bbq_dataset_with_embeddings.csv"
df = pd.read_csv(FILE_PATH)
y = df.performance
X = df.drop(columns=['text', 'performance'])

### isolation forest method

In [47]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest()
clf.fit(X)

feature_importance = diffi_score(clf, X)

D:\FinalProject\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ExtraTreeRegressor was fitted without feature names
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ExtraTreeRegressor was fitted without feature names
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ExtraTreeRegressor was fitted without feature names
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ExtraTreeRegressor was fitted without feature names
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ExtraTreeRegressor was fitted without feature names
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ExtraTreeRegressor was fitted without feature names
  warnings.warn(
D:\FinalProject\Lib\site-pac

In [48]:
indecies = np.argwhere(feature_importance>10000)

In [49]:
indecies = list(indecies.flatten())

### lingam

In [51]:
df = df.drop(columns=['text'])

In [52]:
columns = range(0,len(df.columns))

In [53]:
from causallearn.search.FCMBased import lingam
feature_importance = []

for i in range((len(columns)//100) + 1):
    model = lingam.ICALiNGAM(42, 10)
    untill = min(len(columns), (1+(i+1)*100))
    ling=model.fit(df.iloc[:,[columns[0]] + list(columns[(1+i*100):untill])])
    if len(feature_importance) != 0 :
        feature_importance = np.concatenate((feature_importance, model.adjacency_matrix_[0][1:]), axis=0)
    else:
        feature_importance = model.adjacency_matrix_[0][1:]

D:\FinalProject\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
D:\FinalProject\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  

In [54]:
feature_importance = np.concatenate(([0],feature_importance), axis=0)

In [55]:
indecies = np.argwhere(feature_importance!=0)
indecies = list(indecies.flatten())

### new dataframe

In [56]:
#new df

df_isolation = X.iloc[:,list(indecies)]

In [57]:
df_isolation

,73,77,133,158,160,169,192,198,317,342,...,650,653,654,659,675,680,683,685,691,699
0,-0.294704,-3.113355,-0.493643,-0.645881,1.301734,1.874767,-0.210236,1.402020,1.169437,-0.465324,...,0.214770,0.120293,0.392796,0.002180,0.390732,0.247765,1.141313,-0.027745,1.104657,-0.742588
1,-0.169022,-2.480204,-1.296580,-0.570441,1.321237,0.580881,-0.401286,0.953878,1.130438,0.107205,...,-0.139032,-0.377195,0.099636,-0.255388,0.297478,0.891213,0.222010,-0.329855,0.662936,-0.428735
2,0.745646,-2.825164,-0.829045,-0.200379,1.394566,1.343194,-0.513382,0.124924,0.707194,0.056008,...,0.425801,-0.370254,0.401942,0.858001,-0.331172,0.176874,0.167875,0.060203,1.392434,-0.673819
3,0.639917,-2.969395,-0.993295,-0.883805,1.237065,1.391015,-0.135375,0.386089,1.325778,-0.167745,...,-0.146985,-1.290795,-0.408912,0.058352,0.261937,-0.104822,0.425844,0.058671,-0.536961,-1.057293
4,-0.283076,-2.673019,0.273076,-0.757332,1.002035,2.017928,-0.309608,-0.556812,0.304891,0.074373,...,-0.564492,-0.139324,0.607238,-0.498733,-0.174328,1.083966,0.095637,-0.317406,1.202703,-0.073077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,0.328513,-2.088984,-0.562003,-0.388244,1.379567,1.947565,0.026901,1.218782,1.279397,-0.235787,...,0.220268,-0.410581,0.472591,-0.203532,0.737604,0.200827,1.198635,-0.066241,2.023503,-0.745108
4092,-0.180844,-4.401577,-0.002751,0.064402,2.018978,1.852960,-0.498065,1.450371,0.683583,0.097074,...,-0.424484,-0.101326,-0.282693,-0.512365,0.392168,0.854728,0.446684,-0.600306,1.708290,0.307922
4093,0.596401,-3.525992,0.049286,-0.415078,0.581414,1.416348,0.323639,-0.080082,0.704765,0.007401,...,-0.222609,-0.336092,-1.208873,0.375986,0.390222,0.706694,1.252699,0.651163,0.586765,-0.292586
4094,0.424477,-2.674841,-1.310277,-0.424335,0.798124,1.802895,0.278769,0.751669,0.843048,-0.697343,...,0.222746,0.158392,-0.248261,0.365666,0.106567,0.354771,0.111303,0.259274,0.856542,0.040793


In [ ]:
%%time
places = squeezer_parallel(df.to_numpy(),0.4)

num of instances: 4096
instance 1
